#### Transportation Problem

In [1]:
import sys
sys.setrecursionlimit(10000)

# Model (search problem)
class TransportationProblem(object):
    def __init__(self, N,weights):
        # N = number of blocks
        self.N = N
        self.weights=weights

    def startState(self):
        return 1

    def isEnd(self, state):
        return state == self.N

    def succAndCost(self, state):
        # return list of (action, newState, cost) triples
        result = []
        if state + 1 <= self.N:
            result.append(('walk', state + 1, self.weights['walk']))  # Walking cost: 1
        if state + 3 <= self.N:
            result.append(('run', state + 3, self.weights['run']))  # Running cost: 1.5
        if state * 2 <= self.N:
            result.append(('tram', state * 2, self.weights['tram']))  # Tram cost: 2
        return result

In [2]:
def printSolution(solution):
    totalCost, history = solution
    print('totalCost: {}'.format(totalCost))
    for item in history:
        print(item)


#### UCS

In [3]:
import heapq

def uniformCostSearch(problem):
    heap = [(0, problem.startState(), [])]
    bestCost = float('inf')
    bestPath = []

    while heap:
        totalCost, state, history = heapq.heappop(heap)

        if problem.isEnd(state) and len(history) > 0:
            if totalCost < bestCost:
                bestCost = totalCost
                bestPath = history
            continue

        for action, newState, cost in problem.succAndCost(state):
            newCost = totalCost + cost
            heapq.heappush(heap, (newCost, newState, history + [(action, newState, cost)]))

    return (bestCost, bestPath)



In [4]:
def predict(N,weights):
    #     Input (x) : N (number of blocks)
    #     Output (y) : path (sequence of actions) 
    problem= TransportationProblem(N,weights)
    totalCost, history= uniformCostSearch(problem)
    return [action for action, newState, cost in history]


def generateExamples():
    trueWeights= {'walk': 1, 'run': 1.5,'tram': 2}
    return [(N, predict(N, trueWeights)) for N in range(1,20)]

def structuredPerceptron(examples):
    weights = {'walk':0, 'run':0 , 'tram':0}
    for t in range(100):
        numMistakes = 0
        for N, trueActions in examples:
            predActions = predict(N, weights)
            #predActions is y` , trueActions is y
            if predActions != trueActions:
                numMistakes +=1
            for action in trueActions:
                weights[action] -= 1
            for action in predActions:
                weights[action] +=1
        print(f"Iteration {t}, numMistakes= {numMistakes}, weights= {weights}")
        if numMistakes == 0:
            break
        
        
    
examples = generateExamples()

print("training dataset: ")
for example in examples:
    print(' ', example)
    
structuredPerceptron(examples)

training dataset: 
  (1, [])
  (2, ['walk'])
  (3, ['walk', 'walk'])
  (4, ['run'])
  (5, ['run', 'walk'])
  (6, ['run', 'walk', 'walk'])
  (7, ['run', 'run'])
  (8, ['run', 'tram'])
  (9, ['run', 'tram', 'walk'])
  (10, ['run', 'run', 'run'])
  (11, ['run', 'tram', 'run'])
  (12, ['run', 'walk', 'walk', 'tram'])
  (13, ['run', 'run', 'run', 'run'])
  (14, ['run', 'run', 'tram'])
  (15, ['run', 'run', 'tram', 'walk'])
  (16, ['run', 'tram', 'tram'])
  (17, ['run', 'run', 'tram', 'run'])
  (18, ['run', 'tram', 'walk', 'tram'])
  (19, ['run', 'tram', 'tram', 'run'])
Iteration 0, numMistakes= 5, weights= {'walk': 1, 'run': 1, 'tram': 1}
Iteration 1, numMistakes= 5, weights= {'walk': 2, 'run': 2, 'tram': 2}
Iteration 2, numMistakes= 4, weights= {'walk': 2, 'run': 2, 'tram': 3}
Iteration 3, numMistakes= 3, weights= {'walk': 5, 'run': 2, 'tram': 3}
Iteration 4, numMistakes= 7, weights= {'walk': 5, 'run': 4, 'tram': 4}
Iteration 5, numMistakes= 5, weights= {'walk': 5, 'run': 5, 'tram': 5}
Ite